### Fetch Million Song DB Songs and Artists

In [39]:
import os
import sys
import time
import glob
import datetime
import sqlite3
import tables
import pandas as pd 
import numpy as np 

# path to the Million Song Dataset subset (uncompressed)
msd_subset_path='MillionSongSubset'
msd_subset_data_path=os.path.join(msd_subset_path,'data')
msd_subset_addf_path=os.path.join(msd_subset_path,'AdditionalFiles')
assert os.path.isdir(msd_subset_path),'wrong path' 

conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))
q = "SELECT title, artist_name FROM songs"
all_songs = pd.read_sql_query(q, conn)

In [5]:
all_songs.head()

,title,artist_name
0,,Sébastien Roch
1,-start ID-,The Mad Capsule Markets
2,'57 Chevrolet,Billie Jo Spears
3,'Round the Wheel,DJ Harry
4,'Ti Monde (LP Version),BeauSoleil


### Query Spotify API

In [102]:


# def spotify_request(url, token, params):
#     """Make a Spotify request using the specified token and parameters."""
#     r = requests.get(url, headers={'Authorization': 'Bearer {}'.format(token['access_token'])},
#                      params=params)
#     return r.json()

# def build_query(song_df):
#     # drop missing rows
#     song_df.replace(r'', np.nan, regex=True, inplace=True)
#     song_df.dropna(inplace=True)
    
#     query_terms = []
#     for index, row in song_df.iterrows():
#         query_terms.append((quote(row[1]), quote(row[0])))
#     return query_terms

# # get track id by querying song and artist
# def track_id(track, artist):
#     r = requests.get("https://api.spotify.com/v1/search?q=track:{0}%20artist:{1}&type=track".format(track, artist), 
#                      headers={'Authorization': 'Bearer {}'.format(token['access_token'])})
#     if r.status_code == 200:
#         return r.json()
#     else:
#         raise IOError('Spotify API request unsuccessful. Status code:{}'.format(r.status_code))

# # get track analysis
# def track_info(id):
#     r = requests.get("https://api.spotify.com/v1/audio-analysis/{0}".format(id), 
#                      headers={'Authorization': 'Bearer {}'.format(token['access_token'])})
#     return r.json()

OSError: Spotify API request unsuccessful. Status code:401

In [12]:
secrets = load_secrets()
token = get_token(secrets)

In [91]:
track_ids = []
not_found = []
for song in track_and_artist[:100]:
    try:
        track_ids.append(track_id(song[0], song[1])['tracks']['items'][0]['id'])
    except IOError as e:
#         print (e)
        if e.message[-3:] == '401':
            not_found.append(song)


Spotify API request unsuccessful. Status code:401
Spotify API request unsuccessful. Status code:401
Spotify API request unsuccessful. Status code:401
Spotify API request unsuccessful. Status code:401
Spotify API request unsuccessful. Status code:401
Spotify API request unsuccessful. Status code:401
Spotify API request unsuccessful. Status code:401
Spotify API request unsuccessful. Status code:401
Spotify API request unsuccessful. Status code:401
Spotify API request unsuccessful. Status code:401
Spotify API request unsuccessful. Status code:401
Spotify API request unsuccessful. Status code:401
Spotify API request unsuccessful. Status code:401
Spotify API request unsuccessful. Status code:401
Spotify API request unsuccessful. Status code:401
Spotify API request unsuccessful. Status code:401
Spotify API request unsuccessful. Status code:401
Spotify API request unsuccessful. Status code:401
Spotify API request unsuccessful. Status code:401
Spotify API request unsuccessful. Status code:401


In [90]:
len(not_found)

31

In [86]:
track_data = {}
for ident in track_ids:
    track_data[ident] = track_info(ident)

In [92]:
len(track_data.keys())

69

In [64]:
import json
json.dumps(track_data['0nbm4bvneAGlrp5wZKRHT8']).__len__()

310460

### Reading in Tracks from Text File

In [53]:
song_df = pd.read_csv('unique_tracks.txt', 
                      sep="<SEP>", 
                      header=None, 
                      engine='python', 
                      names=[0,1,'artist', 'track'])[['artist','track']]

In [59]:
song_df.head()

,artist,track
0,Faster Pussy cat,Silent Night
1,Karkkiautomaatti,Tanssi vaan
2,Hudson Mohawke,No One Could Ever
3,Yerba Brava,Si Vos Querés
4,Der Mystic,Tangle Of Aspens


In [61]:
queries = build_query(song_df)

In [62]:
queries

[('Silent%20Night', 'Faster%20Pussy%20cat'),
 ('Tanssi%20vaan', 'Karkkiautomaatti'),
 ('No%20One%20Could%20Ever', 'Hudson%20Mohawke'),
 ('Si%20Vos%20Quer%C3%A9s', 'Yerba%20Brava'),
 ('Tangle%20Of%20Aspens', 'Der%20Mystic'),
 ('Symphony%20No.%201%20G%20minor%20%22Sinfonie%20Serieuse%22/Allegro%20con%20energia',
  'David%20Montgomery'),
 ('We%20Have%20Got%20Love', 'Sasha%20/%20Turbulence'),
 ('2%20Da%20Beat%20Ch%27yall', 'Kris%20Kross'),
 ('Goodbye', 'Joseph%20Locke'),
 ('Mama_%20mama%20can%27t%20you%20see%20%3F',
  'The%20Sun%20Harbor%27s%20Chorus-Documentary%20Recordings'),
 ('L%27antarctique', '3%20Gars%20Su%27l%20Sofa'),
 ('El%20hijo%20del%20pueblo', 'Jorge%20Negrete'),
 ('Cold%20Beer%20feat.%20Prince%20Metropolitan', 'Danny%20Diablo'),
 ('Pilots', 'Tiger%20Lou'),
 ('N%20Gana', 'Waldemar%20Bastos'),
 ('006', 'Lena%20Philipsson'),
 ('%28Looking%20For%29%20The%20Heart%20Of%20Saturday', 'Shawn%20Colvin'),
 ('Ethos%20of%20Coercion', 'Dying%20Fetus'),
 ('Rock-N-Rule', 'Emery'),
 ('La%20bo

In [68]:
# bz2 compression
import bz2
data = json.dumps(track_data['0nbm4bvneAGlrp5wZKRHT8']).encode('utf-8')

In [69]:
c_data=bz2.compress(data)

In [70]:
len(c_data)

80815

In [72]:
# bz2.decompress(c_data)

In [73]:
track_data['0nbm4bvneAGlrp5wZKRHT8']['meta']

{'analysis_time': 7.42932,
 'analyzer_version': '4.0.0',
 'detailed_status': 'OK',
 'input_process': 'libvorbisfile L+R 44100->22050',
 'platform': 'Linux',
 'status_code': 0,
 'timestamp': 1444734951}

ModuleNotFoundError: No module named 'pymongo'

In [75]:
!pip install pymongo

    100% |████████████████████████████████| 307kB 1.8MB/s ta 0:00:01


In [76]:
import pymongo 

In [77]:
client = pymongo.MongoClient()
db = client['song_db']

In [79]:
col = db['songs']

In [80]:
col.insert_one(track_data['0nbm4bvneAGlrp5wZKRHT8'])

In [82]:
col.stats

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'song_db'), 'songs.stats')